In [ ]:
import numpy as np 
import pandas as pd
# import os
# import matplotlib.pyplot as plt
# from PIL import Image, ImageDraw, ImageEnhance
# from tqdm.notebook import tqdm
from tqdm import tqdm

# import cv2
# import re
# import time

import sys
sys.path.append('../')
from retinanet import coco_eval
from retinanet import csv_eval
from retinanet import model
# from retinanet import retina
from retinanet.dataloader import *
from retinanet.anchors import Anchors
from retinanet.losses import *
from retinanet.scheduler import *
from retinanet.parallel import DataParallelModel, DataParallelCriterion

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder

#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.optim import Adam, lr_scheduler
import torch.optim as optim


In [2]:
device = torch.device('cpu')
# device = torch.device('cuda')
retinanet = model.resnet101(num_classes=2, device=device)
# retinanet.to(device)

num_features_in of ResidualAfterFPN : 256


In [3]:
anchor = Anchors()
img = torch.randn([4,3,320,320])
img.shape
anchors = anchor(img)

In [4]:
anchors.shape

torch.Size([1, 2000, 4])

In [5]:
anchors

tensor([[[-12., -12.,  20.,  20.],
         [ -4., -12.,  28.,  20.],
         [  4., -12.,  36.,  20.],
         ...,
         [248., 280., 312., 344.],
         [264., 280., 328., 344.],
         [280., 280., 344., 344.]]])

In [6]:
# myan = anchors[:,500:510]
# myan = torch.tensor([[[148.,  84., 180., 116.],
#          [156.,  84., 188., 116.],
#          [164.,  84., 196., 116.],
#          [172.,  84., 204., 116.],
#          [180.,  84., 212., 116.],
#          [188.,  84., 220., 116.],
#          [196.,  84., 228., 116.],
#          [204.,  84., 236., 116.],
#          [212.,  84., 244., 116.],
#          [260.,  84., 292., 116.]]])
# a = myan[0,:,:]

In [11]:
annotation = torch.tensor([[113, 137, 1.37e+02, 159, 0.],
        [140, 82, 182, 120, 0.],
        [142, 250, 170, 235, 0.],
        [200, 72, 242, 110, 1.0000e+00],
        [160, 72, 202, 115, 0.]])
annotation

tensor([[113., 137., 137., 159.,   0.],
        [140.,  82., 182., 120.,   0.],
        [142., 250., 170., 235.,   0.],
        [200.,  72., 242., 110.,   1.],
        [160.,  72., 202., 115.,   0.]])

In [12]:
annotation[:, :4]

tensor([[113., 137., 137., 159.],
        [140.,  82., 182., 120.],
        [142., 250., 170., 235.],
        [200.,  72., 242., 110.],
        [160.,  72., 202., 115.]])

In [20]:
IoU = paps_calc_iou(anchors[0, :,:], annotation[:, :4])
torch.where(IoU > 0.5  )

(tensor([ 427,  462,  463,  466,  467,  468,  499,  500,  502,  503,  539,  540,
          695,  696,  734,  735,  736,  775,  776, 1691, 1693, 1694, 1709, 1710,
         1710, 1711, 1712, 1712, 1713, 1714, 1729, 1730, 1730, 1731, 1732, 1732,
         1733, 1734, 1749, 1750, 1766, 1767, 1768, 1786, 1787, 1788, 1806, 1807,
         1808]),
 tensor([3, 4, 4, 3, 3, 3, 1, 1, 4, 4, 1, 1, 0, 0, 0, 0, 0, 0, 0, 4, 3, 3, 1, 1,
         4, 4, 3, 4, 3, 3, 1, 1, 4, 4, 3, 4, 3, 3, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0]))

In [22]:
IoU[420:]

tensor([[0.0000, 0.3214, 0.0000, 0.0000, 0.2784],
        [0.0000, 0.2554, 0.0000, 0.0000, 0.4086],
        [0.0000, 0.1721, 0.0000, 0.0000, 0.4797],
        ...,
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])

In [23]:
IoU_max, IoU_argmax = torch.max(IoU, dim=1) # num_anchors x 1


In [25]:
IoU_max[420:]

tensor([0.3214, 0.4086, 0.4797,  ..., 0.0000, 0.0000, 0.0000])

In [26]:
IoU_argmax[420:]

tensor([1, 4, 4,  ..., 0, 0, 0])

In [29]:
classification = torch.rand([2000,2])
classification

tensor([[0.7516, 0.8439],
        [0.9287, 0.5158],
        [0.1941, 0.1921],
        ...,
        [0.6719, 0.3492],
        [0.3701, 0.0932],
        [0.2555, 0.8038]])

In [57]:
targets = torch.ones(classification.shape) * -1
targets

tensor([[-1., -1.],
        [-1., -1.],
        [-1., -1.],
        ...,
        [-1., -1.],
        [-1., -1.],
        [-1., -1.]])

In [55]:
torch.where(IoU_max < 0.3)

(tensor([   0,    1,    2,  ..., 1997, 1998, 1999]),)

In [54]:
torch.lt(IoU_max, 0.3)

tensor([True, True, True,  ..., True, True, True])

In [60]:
targets[torch.lt(IoU_max, iou_threshold), :]

tensor([[-1., -1.],
        [-1., -1.],
        [-1., -1.],
        ...,
        [-1., -1.],
        [-1., -1.],
        [-1., -1.]])

In [61]:
iou_threshold = 0.3
targets[torch.lt(IoU_max, iou_threshold), :] = 0
targets[418:]

tensor([[ 0.,  0.],
        [-1., -1.],
        [-1., -1.],
        ...,
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]])

In [62]:
positive_indices = torch.ge(IoU_max, iou_threshold + 0.1)
positive_indices

tensor([False, False, False,  ..., False, False, False])

In [64]:
num_positive_anchors = positive_indices.sum()
num_positive_anchors

tensor(69)

In [65]:
IoU_argmax

tensor([0, 0, 0,  ..., 0, 0, 0])

In [66]:
assigned_annotations = annotation[IoU_argmax, :]
assigned_annotations

tensor([[113., 137., 137., 159.,   0.],
        [113., 137., 137., 159.,   0.],
        [113., 137., 137., 159.,   0.],
        ...,
        [113., 137., 137., 159.,   0.],
        [113., 137., 137., 159.,   0.],
        [113., 137., 137., 159.,   0.]])

In [67]:
assigned_annotations.shape

torch.Size([2000, 5])

In [43]:
assigned_annotations.shape

torch.Size([2000, 5])

In [68]:
# torch.where(targets > -0.5  )

In [69]:
# a = torch.arange(10)
# a = a.reshape(2,5)
# a

In [71]:
te = torch.where(a%2==0)
te

(tensor([0, 0, 0, 1, 1]), tensor([0, 2, 4, 1, 3]))

In [78]:
# torch.where(targets==-1)

(tensor([ 382,  382,  383,  383,  386,  386,  387,  387,  388,  388,  419,  419,
          420,  420,  424,  424,  425,  425,  429,  429,  458,  458,  464,  464,
          465,  465,  504,  504,  509,  509,  538,  538,  542,  542,  543,  543,
          547,  547,  579,  579,  580,  580,  655,  655,  656,  656,  697,  697,
          733,  733,  737,  737,  777,  777,  815,  815,  816,  816, 1689, 1689,
         1695, 1695, 1746, 1746, 1747, 1747, 1748, 1748, 1752, 1752, 1753, 1753,
         1754, 1754, 1769, 1769, 1789, 1789, 1809, 1809]),
 tensor([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
         0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
         0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
         0, 1, 0, 1, 0, 1, 0, 1]))

In [81]:
assigned_annotations

tensor([[113., 137., 137., 159.,   0.],
        [113., 137., 137., 159.,   0.],
        [113., 137., 137., 159.,   0.],
        ...,
        [113., 137., 137., 159.,   0.],
        [113., 137., 137., 159.,   0.],
        [113., 137., 137., 159.,   0.]])

In [82]:
positive_indices

tensor([False, False, False,  ..., False, False, False])

In [75]:
targets[positive_indices, :] = 0

In [83]:
targets[positive_indices, assigned_annotations[positive_indices, 4].long()] = 1
targets

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]])

In [84]:
torch.where(targets==1)

(tensor([ 421,  422,  423,  426,  427,  428,  459,  460,  461,  462,  463,  466,
          467,  468,  469,  498,  499,  500,  501,  502,  503,  506,  507,  508,
          539,  540,  541,  694,  695,  696,  734,  735,  736,  774,  775,  776,
         1690, 1691, 1692, 1693, 1694, 1708, 1709, 1710, 1711, 1712, 1713, 1714,
         1715, 1728, 1729, 1730, 1731, 1732, 1733, 1734, 1735, 1749, 1750, 1751,
         1766, 1767, 1768, 1786, 1787, 1788, 1806, 1807, 1808]),
 tensor([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1,
         1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [85]:
alpha_factor = torch.ones(targets.shape) * 0.25
alpha_factor

tensor([[0.2500, 0.2500],
        [0.2500, 0.2500],
        [0.2500, 0.2500],
        ...,
        [0.2500, 0.2500],
        [0.2500, 0.2500],
        [0.2500, 0.2500]])

In [86]:
alpha_factor = torch.where(torch.eq(targets, 1.), alpha_factor, 1. - alpha_factor)
alpha_factor

tensor([[0.7500, 0.7500],
        [0.7500, 0.7500],
        [0.7500, 0.7500],
        ...,
        [0.7500, 0.7500],
        [0.7500, 0.7500],
        [0.7500, 0.7500]])

In [87]:
torch.where(alpha_factor==0.25)

(tensor([ 421,  422,  423,  426,  427,  428,  459,  460,  461,  462,  463,  466,
          467,  468,  469,  498,  499,  500,  501,  502,  503,  506,  507,  508,
          539,  540,  541,  694,  695,  696,  734,  735,  736,  774,  775,  776,
         1690, 1691, 1692, 1693, 1694, 1708, 1709, 1710, 1711, 1712, 1713, 1714,
         1715, 1728, 1729, 1730, 1731, 1732, 1733, 1734, 1735, 1749, 1750, 1751,
         1766, 1767, 1768, 1786, 1787, 1788, 1806, 1807, 1808]),
 tensor([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1,
         1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [94]:
import torchvision.models as models
resnet101 = models.resnet101(
    progress=False, pretrained=True)

tensor([ 4.2431e-08,  1.4738e-01,  8.4835e-02,  1.3761e-08,  1.6518e-06,
         1.3687e-01,  1.2218e-01,  1.0812e-01, -4.9020e-08,  2.6627e-01,
         1.5964e-01,  1.3127e-01,  4.7342e-08,  1.1993e-01,  6.1626e-09,
        -6.0274e-09,  1.2445e-06,  1.2616e-01,  1.4407e-01,  1.3228e-01,
         4.7165e-08,  1.0532e-01,  1.2379e-05,  2.7342e-05,  7.3187e-09,
         8.8643e-04,  5.9950e-05,  1.4459e-01,  2.9141e-09,  9.4266e-02,
         1.3379e-08,  9.6290e-02,  2.1684e-08,  9.1433e-09,  1.1874e-01,
         1.2786e-01,  1.2977e-01,  1.9630e-08,  1.3666e-01, -2.4055e-08,
         9.5951e-02,  1.0044e-01,  1.8209e-08,  2.2945e-07, -1.2379e-08,
         3.5723e-08,  1.4637e-01,  1.0493e-08,  2.6085e-04,  3.3908e-05,
         8.2383e-02,  5.5850e-09,  1.0825e-01,  1.1816e-01,  1.1051e-01,
         1.0369e-01,  1.1480e-01,  1.0413e-01,  1.5651e-01,  8.8714e-02,
         1.9953e-01,  4.5951e-02,  7.3443e-02,  1.0333e-01])

In [95]:
resnet101.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 